In [23]:
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Bidirectional, Concatenate
import numpy as np

In [6]:
batch_size = 64
epochs = 20
latent_dim = 256
num_samples = 5000

In [7]:
data_path = 'kor.txt'

Read and Prepare the Data :
    - read data from file
    - tokenize characters
    - prepare input and output sequences

In [29]:
input_texts = []
output_texts = []
input_characters = set()
output_characters = set()
with open(data_path, 'r', encoding = 'utf-8') as f:
    lines = f.read().split('\n')
    print("Number of lines read:", len(lines))
for line in lines[:min(num_samples,len(lines)-1)]:
    input_text, output_text, _ = line.split('\t')
    
    # We use 'tab' as the 'start sequence' character
    # for the targets, and '\n' as the 'end sequence' character.
    
    output_text = '\t' + output_text + '\n'
    input_texts.append(input_text)
    output_texts.append(output_text)
    
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in output_text:
        if char not in output_characters:
            output_characters.add(char)

input_characters = sorted(list(input_characters))
output_characters = sorted(list(output_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(output_characters)
max_encoder_seq_length = max([len(text) for text in input_texts])
max_decoder_seq_length = max([len(text) for text in output_texts])

Number of lines read: 5891


In [30]:
print("Number of input texts:", len(input_texts))
print("Number of output texts:", len(output_texts))
#print(output_characters)
print("Number of encoder tokens:", num_encoder_tokens)
print("Number of decoder tokens:", num_decoder_tokens)

Number of input texts: 5000
Number of output texts: 5000
Number of encoder tokens: 72
Number of decoder tokens: 961


Create Token Dictionary for Indexing

In [31]:
input_token_index = dict(
    [(char,i) for i,char in enumerate(input_characters)])
output_token_index = dict(
    [(char,i) for i,char in enumerate(output_characters)])

Data Preparation : 
    - Initisalise Numpy Arrays for :
            - Encoder Input, Decoder Input and Decoder Output

In [32]:
num_pairs = len(input_texts)
max_english_sentence_length = max_encoder_seq_length
max_french_sentence_length = max_decoder_seq_length
num_english_characters = num_encoder_tokens
Num_french_characters = num_decoder_tokens
encoder_input_data = np.zeros(
    (len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype = 'float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length , num_decoder_tokens), dtype = 'float32')
decoder_output_data = np.zeros(
    (len(input_texts), max_decoder_seq_length,num_decoder_tokens) , dtype = 'float32')

Data Encoding : one-hot encoded vectors

In [33]:
for i,(input_text,output_text) in enumerate(zip(input_texts,output_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]] = 1.
    encoder_input_data[i,t+1:,input_token_index[' ']] = 1.
    
    for t, char in enumerate(output_text):
        # decoder_output_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i,t,output_token_index[char]] = 1.
        if t>0:
            # decoder_output_data will be ahead by one timestep
            # and will not include the start character
            decoder_output_data[i,t-1,output_token_index[char]] = 1.
    decoder_input_data[i,t+1:, output_token_index[' ']] = 1.
    decoder_output_data[i , t: , output_token_index[' ']] = 1.

Define Encoder : 
    - unidirectional LSTM

In [34]:
# define an input sequence and process it:
encoder_inputs = Input(shape = (None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state= True)
encoder_outputs,state_h,state_c = encoder(encoder_inputs)
# we discard 'encoder_outputs' and only keep the states.
encoder_states = [state_h,state_c]

Deifne Decoder : - unidirectional LSTM

In [18]:
# set up the decoder, using encoder_states as initial states:
decoder_inputs= Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim,return_sequences = True, return_state = True)
# We set our decoder to return full output sequences, and to return internal states as well but we don’t use this internal states in the training model,  
#  but we will use them in inference.
decoder_outputs, _ ,_ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens,activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

Define, Compile and Train the Model

In [19]:
# Define the model that will turn
# 'encoder_input_data' & 'decoder_input_data' into 'decoder_output_data'
model = Model([encoder_inputs,decoder_inputs], decoder_outputs)
#Run training:
model.compile(optimizer='rmsprop', loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model.fit([encoder_input_data,decoder_input_data], decoder_output_data,
         batch_size=batch_size,
         epochs = epochs,
         validation_split=0.2)

Epoch 1/20
63/63 [==============================] - 21s 257ms/step - loss: 1.9048 - accuracy: 0.7073 - val_loss: 2.5213 - val_accuracy: 0.5960
Epoch 2/20
63/63 [==============================] - 16s 261ms/step - loss: 1.6124 - accuracy: 0.7199 - val_loss: 2.4623 - val_accuracy: 0.6004
Epoch 3/20
63/63 [==============================] - 16s 247ms/step - loss: 1.5105 - accuracy: 0.7320 - val_loss: 2.3807 - val_accuracy: 0.6127
Epoch 4/20
63/63 [==============================] - 16s 248ms/step - loss: 1.4132 - accuracy: 0.7576 - val_loss: 2.2179 - val_accuracy: 0.6371
Epoch 5/20
63/63 [==============================] - 16s 252ms/step - loss: 1.3195 - accuracy: 0.7674 - val_loss: 2.0855 - val_accuracy: 0.6490
Epoch 6/20
63/63 [==============================] - 16s 250ms/step - loss: 1.2482 - accuracy: 0.7733 - val_loss: 1.9479 - val_accuracy: 0.6591
Epoch 7/20
63/63 [==============================] - 16s 260ms/step - loss: 1.2419 - accuracy: 0.7703 - val_loss: 1.8498 - val_accuracy: 0.6693

In [21]:
# define sampling models:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape = (latent_dim,))
decoder_state_input_c = Input(shape = (latent_dim,))
decoder_input_states = [decoder_state_input_h,decoder_state_input_c]
decoder_outputs,state_h,state_c = decoder_lstm(decoder_inputs, initial_state=decoder_input_states)
decoder_states = [state_h,state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs]+decoder_input_states,
    [decoder_outputs] + decoder_states)
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict(
    (i,char) for char,i in input_token_index.items())
reverse_output_char_index = dict(
    (i,char) for char,i in output_token_index.items())
def decode_sequence(input_seq):
    states_value= encoder_model.predict(input_seq)
    output_seq = np.zeros((1,1,num_decoder_tokens))
    output_seq[0,0,output_token_index['\t']] = 1
    
    stop_condition = False
    decoded_sentences = ''
    while not stop_condition:
        output_tokens,h,c = decoder_model.predict(
            [output_seq]+ states_value)
        
        sampled_token_index = np.argmax(output_tokens[0,-1, :])
        sampled_char = reverse_output_char_index[sampled_token_index]
        decoded_sentences += sampled_char
        if(sampled_char == '\n' or len(decoded_sentences) > max_decoder_seq_length):
            stop_condition = True
        
        #update the target sequence (of length 1):
        output_seq = np.zeros((1,1,num_decoder_tokens))
        output_seq[0,0,sampled_token_index] = 1
        
        states_value = [h,c]
    return decoded_sentences
for seq_index in range(20):
    # take one sequence for trying out decoding:
    input_seq = encoder_input_data[seq_index:seq_index+1]
    decoded_sentences = decode_sequence(input_seq)
    print('ProjectGurukul Project: English to Korean Translation ')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:' , decoded_sentences)

1/1 [==============================] - 0s 32ms/step
ProjectGurukul Project: English to Korean Translation 
Input sentence: Go.
Decoded sentence: 누군가 있어.

1/1 [==============================] - 0s 36ms/step
ProjectGurukul Project: English to Korean Translation 
Input sentence: Hi.
Decoded sentence: 그녀는 그 사람은 아직 것을 보다.

1/1 [==============================] - 0s 35ms/step
ProjectGurukul Project: English to Korean Translation 
Input sentence: Run!
Decoded sentence: 이 있어?

1/1 [==============================] - 0s 34ms/step
ProjectGurukul Project: English to Korean Translation 
Input sentence: Run.
Decoded sentence: 이 있어?

1/1 [==============================] - 0s 33ms/step
ProjectGurukul Project: English to Korean Translation 
Input sentence: Who?
Decoded sentence: 우리는 아직도 수 있어?

1/1 [==============================] - 0s 36ms/step
ProjectGurukul Project: English to Korean Translation 
Input sentence: Wow!
Decoded sentence: 우리는 그 사람이 아니다.

1/1 [==============================] - 0s 35ms/step

KeyboardInterrupt: 